In [2]:
import numpy as np
import pandas as pd

# Colaborative Filtering (CF)

## Memory-Based
Utiliza algun tipo de métrica para medir similitud entre Usuarios o Items

### Metricas (Similitudes/Distancias):
#### Similitud de pearson (coeficiente de correlacion)

Se toman todos los items que los usuarios u y a calificaron y se evalúa que tan parecido lo hicieron

$\huge w_{a,u} = \frac{\sum_{i=1}^n{(r_{a,i} - \bar{r_a})(r_{u,i} - \bar{r_u})}}{n\sigma_a \sigma_u} = \frac{\sum_{i=1}^n{(r_{a,i} - \bar{r_a})(r_{u,i} - \bar{r_u})}}{{\sqrt{\sum_{i=1}^n{(r_{a,i} - \bar{r_a})^2}} \sqrt{\sum_{i=1}^n(r_{u,i} - \bar{r_u})^2}}}$

- $w_{a,u}$ similitud o correlacion de pearson entre usuario a y u
- $\sigma_a$ y $\sigma_u$ son los desvios de $a$ y $u$ respectivamente
- $n$ son todos los items que el usuario $u$ y $a$ calificaron

### (Se podria medir similitud entre items de la misma forma)

#### Similitud del coseno
Suponiendo que las medias $\bar{r_u}$ y $\bar{r_a}$ son cero nos queda la similitud del coseno:

$\huge w_{a,u} = \frac{\sum_{i=1}^n{r_{a,i}r_{u,i}}}{{\sqrt{\sum_{i=1}^n{r_{a,i}^2}} \sqrt{\sum_{i=1}^n r_{u,i}^2}}}$

#### Similitud de Jaccard
Se usa en situaciones binarias (Like / dislike)

$\huge J(u_1, u_2) = \frac {\| L_{u_1} \cap L_{u_2} \| + \| D_{u_1} \cap D_{u_2} \| - \| L_{u_1} \cap D_{u_2} \| - \| D_{u_1} \cap L_{u_2} \|} {\|L_{u_1} \cup D_{u_2} \cup L_{u_2} \cup D_{u_1}\|}$

$\huge w_{a,u} = \frac{\sum_{i=1}^n{r_{a,i}r_{u,i}}}{R_a\cup R_u}$
$R_a$ y $R_u$ son la cantidad de items que califico el usuario a y el usuario u respectivamente

#### Otras metricas:

- Distancia Euclidiana
- Manhattan
- Hamming

### Estimación de "probabilidad" de que un usuario haya "likeado/dislikeado"
Esta entre -1 y 1

$\huge P(u_j, i_k) = \frac{\sum_{i=1}^{N_L} J(u_j, u_{i}^k) - \sum_{i=1}^{N_D} J(u_j, u_{i}^k)}{N_L + N_D}$

La posibilidad de que el usuario $u_j$ likee el elemento $i_k$ se define como $P(u_j, i_k)$ donde $N_L$ y $N_D$ son la cantidad de usuarios que likearon y dislikearon el elemento $i_k$ respectivamente. Los $u_i^k$ corresponen a los usuarios que likearon o dislikearon el item $k$

Notar que las dos sumas barren todos los usuarios.  
La primer suma, sumará si hay similitud entre los usuarios que likearon y el usuario en cuestion  
La segunda suma, restará si hay similitud entre los usuarios que dislikearon y el usuario en cuestion  
Siempre todo analizado sobre el mismo item  
Si mi indice de Jaccard con todos los que likearon es 1 y mi indice de Jaccard con todos los que dislikearon es -1 P = 1

### Estimación de rating

$\huge \hat{x}_{k,m} = \frac{\bar{x}_k+\sum_{u_a}sim_u(u_k,u_a)(x_{a,m}-\bar{x}_{u_a})}{\sum_{u_a}|sim_u(u_k,u_a)|}$

- $\hat{x}_{k,m}$ -> Rating estimado del usuario k'esimo a la pelicula m
- $\bar{x}_k$ -> Promedio de ratings del usuario K'esimo (sobre todas la peliculas que calificó)
- $sim_u(u_k,u_a)$ -> Similitud entre usuarios $u_k$ y $u_a$
- $x_{a,m}$ -> Rating que dio el usuario a a la pelicula m

#### Toy example:
Estamos en este ejemplo haciendo la matriz de $n_un_i$, al reves que en CB
- 1 -> Likes
- -1 -> Dislikes
- 0 -> Ni like ni dislike

In [3]:
R = np.array([[ 1, 1, 1, 0, 0, 0],
              [ 0, 0, 0, 0, 1, 0],
              [ 1, 1, 1,-1,-1,-1],
              [-1,-1,-1, 1, 1, 1],
              [ 0, 0, 0, 0, 0, 0],
              [ 0, 0, 0, 1, 1, 0],
              [ 0, 0, 0,-1,-1,-1],
              [ 1, 0, 0, 0,-1, 0],
              [ 1, 1, 0, 0,-1,-1]])

In [4]:
index_1 = 2
index_2 = 8 #3
# Cosine
suma_productos = (R[index_1]*R[index_2]).sum()
modulo_1 = ((R[index_1]**2).sum())**0.5
modulo_2 = ((R[index_2]**2).sum())**0.5
metrica = suma_productos/(modulo_1*modulo_2)
print(suma_productos, modulo_1, modulo_2, metrica)

4 2.449489742783178 2.0 0.8164965809277261


http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
similarities = cosine_similarity(R)

In [7]:
similarities[2,8], similarities[2,3]

(0.8164965809277261, -1.0000000000000002)

In [8]:
def getJaccardSimilarityMatrix(R):
    R_abs = 1.0*(abs(R)>0)
    intersect = R_abs.dot(R_abs.T)
    users_count = R_abs.sum(axis = 1)
    users_count = users_count.reshape(users_count.shape[0],1)
    denom = users_count + users_count.T
    denom = denom - intersect
    denom[denom==0] = 1
    similarity = np.dot(R,R.T)
    similarity = similarity/denom
    return similarity

In [9]:
jaccard_sim = getJaccardSimilarityMatrix(R)

In [10]:
jaccard_sim[2,8], jaccard_sim[2,3]

(0.6666666666666666, -1.0)

In [11]:
def predictions(R, similarityMatOrig, divide_by_weights_sum = True, count_diag = False, means = 0):
    # divide_by_weights_sum -> Divide por la suma de los pesos y no por la cantidad de elementos likeados/dislikeados
    similarityMat = similarityMatOrig.copy()
    if not count_diag:
        np.fill_diagonal(similarityMat,0)
    difMat = (R-means).T.dot(similarityMat).T
    if divide_by_weights_sum:
        denomin = abs(similarityMat)[:,::-1].sum(axis = 1)
    else: 
        denomin = abs(R.T).sum(axis=1)
    denomin[denomin == 0] = 1
    nomalizer = abs(R.T).sum(axis=1)
    nomalizer[nomalizer == 0] = 1
    if divide_by_weights_sum:
        result = (difMat.T/denomin).T
    else:
        result = difMat/denomin
    result = result + means
    return result

In [12]:
calculated_prediction = predictions(R, jaccard_sim)

In [13]:
calculated_prediction[0,5], calculated_prediction[8,2], calculated_prediction[8,3]

(-0.8484848484848485, 0.5621621621621622, -0.627027027027027)

### Evaluación
- Medir accuracy en datos de testing
- Definiendo un TOP-K, vemos que porcentaje de las veces, algo que se likeo entra dentro de ese top K en mi sistema de recomendación

![comparacion_de_metricas_rec_sys.png](comparacion_de_metricas_rec_sys.png)

Se puede ver que para el caso de "Pearson not train", el 37% de las veces, el item relevante aparece dentro del 2% top list. Es decir, el 37% de las veces el item esta dentro de los primeros 20 items recomendados. 

![comparacion_de_metricas_rec_sys_2.png](comparacion_de_metricas_rec_sys_2.png)

**Ejemplo movielens**

! wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

In [ ]:
! unzip ml-100k.zip

![BLOG_CCA_8.png](BLOG_CCA_8.png)

**Cargo dataset**

In [19]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))
df[:4]

Number of users = 943 | Number of movies = 1682


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923


**Separo test de train y armo matrices**

In [20]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

In [21]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [22]:
print(test_data_matrix.shape)
print(train_data_matrix.shape)
train_data_matrix

(943, 1682)
(943, 1682)


array([[0., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

### User-Item

![BLOG_CCA_11.png](BLOG_CCA_11.png)

$\huge \hat{x}_{k,m} = \frac{\bar{x}_k+\sum_{u_a}sim_u(u_k,u_a)(x_{a,m}-\bar{x}_{u_a})}{\sum_{u_a}|sum_u(u_k,u_a)|}$

In [44]:
# Calculo media solo teniendo en cuando los distintos de cero
mu = train_data_matrix[train_data_matrix.nonzero()].mean()

In [50]:
user_similarity = cosine_similarity(train_data_matrix)
print(user_similarity)
print(user_similarity.shape)

[[1.         0.13982086 0.04161716 ... 0.12988924 0.13040404 0.3362147 ]
 [0.13982086 1.         0.09874805 ... 0.16269861 0.0990332  0.09777028]
 [0.04161716 0.09874805 1.         ... 0.04993324 0.15207632 0.02103247]
 ...
 [0.12988924 0.16269861 0.04993324 ... 1.         0.07174014 0.05891074]
 [0.13040404 0.0990332  0.15207632 ... 0.07174014 1.         0.11633849]
 [0.3362147  0.09777028 0.02103247 ... 0.05891074 0.11633849 1.        ]]
(943, 943)


In [55]:
# Podemos ver con esto que usuarios se parecen más entre si
sorted_users = np.argsort(user_similarity, axis = 1).T[::-1].T
print(sorted_users)

[[  0 456 822 ... 687 272 240]
 [  1 700 930 ... 171 669 179]
 [  2 723 783 ... 589 348 224]
 ...
 [940 816 688 ... 146 425 315]
 [941 779  90 ... 661  33 691]
 [942 631 681 ... 630 625 588]]


In [63]:
print('El usuario {} es el mas similar al usuario 0 y tiene una similitud de {}'.format(sorted_users[0, 1], user_similarity[0, sorted_users[0, 1]]))
print('El usuario {} es el mas similar al usuario 1 y tiene una similitud de {}'.format(sorted_users[1, 1], user_similarity[1, sorted_users[1, 1]]))

El usuario 456 es el mas similar al usuario 0 y tiene una similitud de 0.44309547517490067
El usuario 700 es el mas similar al usuario 1 y tiene una similitud de 0.493090272341686


In [64]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [65]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')+mu

In [66]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [67]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))

User-based CF RMSE: 1.3805835622679588
User-based CF RMSE: 1.360518049957164


### Item-Item

![BLOG_CCA_10.png](BLOG_CCA_10.png)

$\huge \hat{x}_{k,m} = \frac{\sum_{i_b}sim_i(i_m,i_b)x_{k,b}}{\sum_{i_b}|sum_i(i_m,i_b)|}$